In [15]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sortedcontainers import SortedDict


In [12]:
class Point:
    id = 1
    def __init__(self,a,b,c = 0):
        self.id = Point.id
        Point.id+=1
        
        self.x = a
        self.y = b
        self.classe = c

    def __add__(self, other):
        if isinstance(other, Point):
            return Point(self.x+other.x, self.y+other.y)
        else:
            raise TypeError("Unsupported operand type")
        
    def __sub__(self, other):
        if isinstance(other, Point):
            return Point(self.x-other.x, self.y-other.y)
        else:
            raise TypeError("Unsupported operand type")
        
    def __str__(self):
        return str(self.id)+": "+str(self.x)+"   "+str(self.y)
    
    def __lt__(self,other):
        if isinstance(other, Point):
            vec_prod = (self.x*other.y)-(self.y*other.x)
            if vec_prod>0:
                return True
            elif vec_prod<0:
                return False
            else:
                raise Exception("Segments are parallel")
        else:
            raise TypeError("Unsupported operand type")
        
    def Reset():
        Point.id = 1

In [4]:
class Segment:
    def __init__(self,a: Point, b: Point, shake = 1e-7):
        if b<a:
            temp = a
            a = b
            b = temp
        shakes = np.random.uniform(-shake, shake, 4)
        self.bgn = a+Point(shakes[0],shakes[1])
        self.end = b+Point(shakes[2],shakes[3])
        self.slope = (self.end.y-self.bgn.y)/(self.end.x-self.bgn.x)
        self.intercept = self.end.y - (self.slope * self.end.x)
        self.id = 0 #Usado apenas para indicar ordem durante o algoritmo de varredura linear
    
    def Y_At_X(self,x):
        return self.slope*x+self.intercept


    def Invert(self):
        temp = self.bgn
        self.bgn = self.end
        self.end = temp

    def __str__(self):
        return "Begin: "+str(self.bgn.x)+"   "+str(self.bgn.y)+"\nEnd: "+str(self.end.x)+"   "+str(self.end.y)+"\nInclination: "+str(self.inclination)
    
    def __lt__(self,other):
        if isinstance(other, Segment):
            return self.bgn.x < other.bgn.x
        else:
            raise TypeError("Unsupported operand type")

## Primitivas

In [5]:
def Clockwise(a: Segment, b: Segment): #returns true if a is clockwise to b
    frst_seg = a.end - a.bgn
    scnd_seg = b.end - b.bgn
    vec_prod = (frst_seg.x*scnd_seg.y)-(frst_seg.y*scnd_seg.x)
    if vec_prod>0:
        return True
    elif vec_prod<0:
        return False
    else:
        raise Exception("Segments are parallel")

In [6]:
def SegmentsIntercept(a: Segment,b: Segment):
    check_one = Clockwise(a,Segment(a.bgn,b.bgn))
    check_two = Clockwise(a,Segment(a.bgn,b.end))
    if check_one==check_two:
        return False
    check_one = Clockwise(b,Segment(b.bgn,a.bgn))
    check_two = Clockwise(b,Segment(b.bgn,a.end))
    if check_one==check_two:
        return False
    return True

## Data reading and pre-processing

In [7]:
def SetTwoDimensions(df):
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(df)
    df_pca = pd.DataFrame(data = pca_result, columns = ['x', 'y'])
    return df_pca

In [8]:
def PreProcessData(path, tgtCol):
    df = pd.read_csv(path)
    tgtData = df.iloc[:, tgtCol]  # Save the nth column
    df = df.drop(df.columns[tgtCol], axis=1)
    df = SetTwoDimensions(df)
    df['target_data'] = tgtData
    return df

In [ ]:
def ExtractPoints(df):
    rb_tree_points = SortedDict()
    for i in range(len(df)):
        p = Point(df.loc[i, "x"], df.loc[i, "y"], df.loc[i,"target_data"])
        rb_tree_points[p] = None
    return rb_tree_points


In [ ]:
def ReadData(path, tgtCol):
    df = PreProcessData(path, tgtCol)
    rb_tree_points = ExtractPoints(df)
    x_bound = df['x'].max()
    return rb_tree_points, x_bound


## Convex Hull

In [19]:
def PointInsideHull(p, hull_seg, x_bound):
    s = Segment(Point(x_bound+1,0),p)
    count = 0
    it_hull_seg = list(hull_seg.itervalues())
    for seg in it_hull_seg:
        if SegmentsIntercept(s,seg):
            count+=1
    if count%2==0:
        return False
    else:
        return True

In [13]:
def ConvexHull(rb_tree_points, x_bound):
    hull_points = SortedDict(key=lambda p: p, cmp=lambda self, other: self.x < other.x)
    hull_seg = SortedDict()
    p0 = rb_tree_points.peekitem(0)[0]
    p1 = rb_tree_points.peekitem(1)[0]
    p2 = rb_tree_points.peekitem(2)[0]
    s0 = Segment(p0,p1)
    s1 = Segment(p1,p2)
    s2 = Segment(p2,p0)
    hull_points[p0] = None
    hull_points[p1] = None
    hull_points[p2] = None
    hull_seg[s0.bgn.x] = s0
    hull_seg[s1.bgn.x] = s1
    hull_seg[s2.bgn.x] = s2
    it_points = list(rb_tree_points.keys())
    for p in it_points[3:]:
        if not PointInsideHull(p,hull_seg,x_bound):
            hull_points[p] = None
            position = hull_points.index(p)
            n = len(hull_points)
            while True: #looks for the first tangent line

        #checks if p is inside the hull
    
    

In [27]:
d = SortedDict()
p1 = Point(1,2)
p2 = Point(3,1)
p3 = Point(2,2)
p4 = Point(2,1)
Point.Reset()
d[0] = 5
d[1] = 6
d[2] = 7
d[3] = 8
for key,v in d.items():
    print(key,d)

0 SortedDict({0: 5, 1: 6, 2: 7, 3: 8})
1 SortedDict({0: 5, 1: 6, 2: 7, 3: 8})
2 SortedDict({0: 5, 1: 6, 2: 7, 3: 8})
3 SortedDict({0: 5, 1: 6, 2: 7, 3: 8})


## Sweep Line

In [9]:
#Para a varredura funcionar, eu estou supondo que o vetor de valores de x à varrer está ordenado e que o vetor de segmentos foi corretamente pré-computado, 
#isto é, o vetor de segmentos deve ser ordenado pela função de ordem já definida, não existem 2 pontos extremos com o mesmo valor de x e o segmento que ocupa a 
#i-ésima posição do vetor de segmentos deve ter id=i

#Devido aos cuidados nescessários com isso, também já comecei a implementar as funções que leem os dados e pré-computam tudo para o SweepLine rodar fino
#PS: QUE ALGORITMO FEIO DO CARALHO

def SweepLine(segments):
    j = 0
    comp = dict()
    ans = dict()
    scope = []
    for i in range(len(x_vector)):
        seg = segment_vector[j]
        #olhar no escopo se algum segmento já alcançado começa em x==i
        if seg.bgn==i:
            scope.append(seg)
            j+=1
            SegmentEnteringComparison(seg,scope,comp)
        #olhar no escopo qual dos segmentos termina em x==i
        else:
            for k in scope:
                if k.end==i:
                    #compara tudo
                    SegmentLeftingComparison(i,seg.id,segment_vector,scope,comp,ans)
                    scope.remove(k)
    return ans

def SegmentEnteringComparison(seg: Segment, scope: List[Segment], comp: Dict[int, Dict[int, bool]]):
    above = dict()
    for i in scope:
        if seg.bgn.y<i.Y_At_X(seg.bgsn.x):
            above[i.id] = False
            comp[i.id][seg.id] = True
        else:
            above[i.id] = True
            comp[i.id][seg.id] = False
    comp[seg.id] = above

def SegmentLeftingComparison(x,
                             seg_indx: int, 
                             segment_vector: List[Segment],  
                             comp: Dict[int, Dict[int, bool]], 
                             ans: Dict[int,List[int]]):
    seg = segment_vector[seg_indx]
    key = seg.id
    for i in comp[key]:
        if comp[key][i] != (seg.end.y>segment_vector[i].Y_At_X(x)):
            ans[key].append(i)
            ans[i].append(key)

In [11]:
p1 = Point(1,2)
p2 = Point(3,1)
p3 = Point(2,2)
p4 = Point(2,1)
Point.Reset()
vec_p = np.array([p1,p2,p3,p4])
vec_s = np.sort(vec_p)
for i in vec_p:
    print(i)

5: 0.9999950042124603   2.0000068779394993
6: 3.000008867663188   0.9999994139122027
7: 1.9999983079803205   1.999997851747353
8: 2.000000983071408   0.9999956488528767


In [30]:
for i in vec_s:
    print(i)

2: 2.999997913822665   1.0000081665137703
4: 2.000008793410377   1.0000003650231482
3: 1.999999953488618   1.999990186458855
1: 0.9999948015166591   1.9999901810797924
